original

In [1]:
import csv
import gzip
import time
import os
import pandas as pd
import re
from numpy import where as npwhere
from functools import reduce

def mapMethytilatedCitosine(metfile,intersection_file_path,output_file_path):
    chrs_data = {}
    with gzip.open(metfile, 'rt') as f:
        for line in f:
            chr_index = line.split('\t')[0]
            coordinate_index = int(line.split('\t')[1])
            try:
                chrs_data[chr_index][coordinate_index] = line[:-5]
            except:
                chrs_data[chr_index] = {coordinate_index:line[:-5]}

    with open(intersection_file_path) as f:
        reader = csv.reader(f, delimiter='\t')
        data = list(reader)

    with open(output_file_path, 'a+') as whateverBox:
        for each_box in data:
            box_areas = {}
            box_start = int(each_box[1])
            box_end = int(each_box[2])
            chr_location = each_box[0]
            sequence_identifier = each_box[3]
            # tenemos que aplicar rango desde una posicion inicial porque es un archivo 0 based, si fuese 1 based el rango estaría mal y tendria que restarle
            # uno al archivo al box start
            box_areas['box'] = list(range(box_start, box_end))

            for boxarea in box_areas:
                for coordinate in box_areas[boxarea]:
                    matchline = chrs_data[chr_location].get(coordinate, False)
                    if matchline:
                        whateverBox.write('{}\t{}\t{}\n'.format(matchline, sequence_identifier, boxarea))
                        

def loadFileRemoveMtsAndLowMetReport(replicate):
        bxdf = pd.read_csv(
            replicate,
            sep='\t', 
            names=['chr', 'position', 'strand', 'mets', 'nomets', 'context', 'boxID', 'side'],
            usecols=['chr', 'position', 'strand', 'boxID','mets', 'nomets', 'context']
                )
        bxdf.drop(bxdf[(bxdf.chr == 'Mt') | (bxdf.chr == 'Pt')].index, inplace=True)
        bxdf = bxdf[bxdf['mets'] + bxdf['nomets'] >= 5]
        return bxdf

def calculateValueAsPercentageOfMetCs(replicatedf,replicanamedf):
    replicatedf[replicanamedf] = replicatedf['mets'] / (replicatedf['nomets'] + replicatedf['mets'])
    replicatedf = replicatedf.drop(columns=['mets','nomets'])
    return replicatedf
def calculateValueAsCualitative(replicatedf,replicanamedf):
    """only zero or one, meaning one that there is methylation"""
    replicatedf[replicanamedf] = npwhere(replicatedf['mets']>replicatedf['nomets'], 1, 0)
    replicatedf = replicatedf.drop(columns=['mets','nomets'])
    return replicatedf

In [5]:
import csv
import gzip
import time
import os
import pandas as pd
import re
from numpy import where as npwhere
from functools import reduce

def mapMethytilatedCitosine(metfile,intersection_file_path,output_file_path):
    chrs_data = {}
    with gzip.open(metfile, 'rt') as f:
        for line in f:
            chr_index = line.split('\t')[0]
            coordinate_index = int(line.split('\t')[1])
            try:
                chrs_data[chr_index][coordinate_index] = line[:-5]
            except:
                chrs_data[chr_index] = {coordinate_index:line[:-5]}

    with open(intersection_file_path) as f:
        reader = csv.reader(f, delimiter='\t')
        data = list(reader)

    with open(output_file_path, 'w') as whateverBox:
        for each_box in data:
            box_areas = {}
            box_start = int(each_box[1])
            box_end = int(each_box[2])
            chr_location = each_box[0]
            sequence_identifier = each_box[3]
            # tenemos que aplicar rango desde una posicion inicial porque es un archivo 0 based, si fuese 1 based el rango estaría mal y tendria que restarle
            # uno al archivo al box start
            box_areas['box'] = list(range(box_start, box_end))

            for boxarea in box_areas:
                for coordinate in box_areas[boxarea]:
                    matchline = chrs_data[chr_location].get(coordinate, False)
                    if matchline:
                        whateverBox.write('{}\t{}\t{}\n'.format(matchline, sequence_identifier, boxarea))
                        

def loadFileRemoveMtsAndLowMetReport(replicate,cgcontext):
        bxdf = pd.read_csv(
            replicate,
            sep='\t', 
            names=['chr', 'position', 'strand', 'mets', 'nomets', 'context', 'boxID', 'side'],
            usecols=['chr', 'boxID','mets', 'nomets', 'context']
                )
        
        bxdf.drop(bxdf[(bxdf.chr == 'Mt') | (bxdf.chr == 'Pt')].index, inplace=True)
        if cgcontext == 'CG':
            bxdf.drop(bxdf[(bxdf.context != 'CG')].index, inplace=True)
        if cgcontext == 'nonCG':
            bxdf.drop(bxdf[(bxdf.context == 'CG')].index, inplace=True)
        
        bxdf = bxdf[bxdf['mets'] + bxdf['nomets'] > 4]
        
        return bxdf

def calculateValueAsWheigtedPercentageOfMetCs(replicatedf=None,replicanamedf=None):
    replicatedf = replicatedf.drop(columns=['chr','context'])
    replicatedf = replicatedf.groupby('boxID', as_index=False).sum()
    replicatedf[replicanamedf] = replicatedf['mets'] / (replicatedf['nomets'] + replicatedf['mets'])
    replicatedf = replicatedf.drop(columns=['mets','nomets'])
    return replicatedf
def calculateValueAsCualitative(replicatedf,replicanamedf):
    """only zero or one, meaning one that there is methylation"""
    replicatedf[replicanamedf] = npwhere(replicatedf['mets']>replicatedf['nomets'], 1, 0)
    replicatedf = replicatedf.drop(columns=['mets','nomets'])
    return replicatedf

In [ ]:
import shutil
def movetofolderallthefiles(origin, destinationfolder):
    shutil.copy2(origin, destinationfolder)

In [4]:
working_folder = '/home/joaquin/projects/methylation/data/bisulfite_quick_and_dirty_rep1_rep2'
intersection_file = '/home/joaquin/projects/methylation/data/commonData/arabidopsisThaliana/intersect/Peaks_centered_56bp.bed'
basedmrfolder = '/home/joaquin/projects/methylation/methylation_jose/met_site'
experimentsClasification = {}
for root, dirs, files in os.walk(working_folder):

    for file in files:
        if file.endswith('CX_report.txt.gz'):
            replicate, hour, condition =root.strip().split('/')[-3:]
            experimentName = hour+condition
            experimentsClasification.setdefault(experimentName, [])
            targetMetFilename = root+'/'+file
            finalnamedestination = basedmrfolder+'/'+experimentName+replicate+'_CX_report.txt.gz'
            outputfile = targetMetFilename[:-3] + 'Peaks_centered_56bp.tsv'
            experimentsClasification[experimentName].append(outputfile)
            print(outputfile)
            mapMethytilatedCitosine(targetMetFilename,intersection_file,outputfile)

            

/home/joaquin/projects/methylation/data/bisulfite_quick_and_dirty_rep1_rep2/rep3/6/ACC/WGBS8_FDLM220151345-1a_H5HGMDSX3_L1_1_val_1_bismark_bt2_pe.multiple.deduplicated.CX_report.txtPeaks_centered_56bp.tsv
True
/home/joaquin/projects/methylation/data/bisulfite_quick_and_dirty_rep1_rep2/rep3/6/Mock/WGBS2_FDLM220151339-1a_H5HGMDSX3_L1_1_val_1_bismark_bt2_pe.multiple.deduplicated.CX_report.txtPeaks_centered_56bp.tsv
True
/home/joaquin/projects/methylation/data/bisulfite_quick_and_dirty_rep1_rep2/rep3/6/JA/WGBS5_FDLM220151342-1a_H5HGMDSX3_L1_1_val_1_bismark_bt2_pe.multiple.deduplicated.CX_report.txtPeaks_centered_56bp.tsv
True
/home/joaquin/projects/methylation/data/bisulfite_quick_and_dirty_rep1_rep2/rep3/24/ACC/WGBS9_FDLM220151346-1a_H5GTJDSX3_L1_1_val_1_bismark_bt2_pe.multiple.deduplicated.CX_report.txtPeaks_centered_56bp.tsv
True
/home/joaquin/projects/methylation/data/bisulfite_quick_and_dirty_rep1_rep2/rep3/24/Mock/WGBS3_FDLM220151340-1a_H5HGMDSX3_L1_1_val_1_bismark_bt2_pe.multiple.de

In [25]:
peaksfile = '/home/joaquin/projects/methylation/data/commonData/arabidopsisThaliana/intersect/allThePossiblePeaksnine.bed'

peaks = pd.read_csv(peaksfile, sep='\t', names=['chrm','start','end','boxID'])
for metcontext in ['CG','nonCG']:  
    allthedata = []
    cgdataframe = []
    noncgdataframes = []
    finaldf = None
    for experimentCondition in experimentsClasification:
        listOfDfReplicatesPercentageOfMetCs = []
        listOfDfReplicatesCualitative = []
        listOfNamesReplicates = []
        df_merged = None
        for replicatePath in experimentsClasification[experimentCondition]:
            replicateDf = None
            replicaname =  experimentCondition+replicatePath.split('/')[-4]
            replicateDf = loadFileRemoveMtsAndLowMetReport(replicatePath, metcontext)

            print(replicateDf)
            replicateDfPercentageOfMetCs = calculateValueAsWheigtedPercentageOfMetCs(replicateDf,replicaname)
            print(replicateDfPercentageOfMetCs)
            listOfNamesReplicates.append(replicaname)
            listOfDfReplicatesPercentageOfMetCs.append(replicateDfPercentageOfMetCs)

        df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['boxID'],
                                                    how='outer'), listOfDfReplicatesPercentageOfMetCs)
        print(df_merged)
#         df_merged = df_merged.dropna()
        df_merged[experimentCondition] = df_merged[listOfNamesReplicates].mean(axis=1)
        print(df_merged)
        df_merged = df_merged.drop(columns=listOfNamesReplicates)
        allthedata.append(df_merged)
    finaldf = reduce(lambda  left,right: pd.merge(left,right,on=['boxID'],
                                                    how='outer'), allthedata)
    finaldf = peaks.merge(finaldf, on='boxID', how='outer')
    finaldf = finaldf[['chrm','start','end','boxID','1Mock','1JA','1ACC','6Mock','6JA','6ACC','24Mock','24JA','24ACC']]



    finaldf.to_csv(f'/home/joaquin/projects/methylation/data/bisulfite_quick_and_dirty_rep1_rep2/{metcontext}_BSMet.tsv',sep='\t', index=False)
    
    
#     finaldf = finaldf.dropna()

         chr  mets  nomets context       boxID
0          1     0      12      CG      1:1538
1          1     0       6      CG      1:1538
36         1     0       8      CG      1:1538
37         1     0       7      CG      1:1538
71         1     0       6      CG      1:1538
...      ...   ...     ...     ...         ...
4420571    5    10       0      CG  5:26974854
4420587    5     7       1      CG  5:26974854
4420588    5     9       1      CG  5:26974854
4420601    5     2       3      CG  5:26974854
4420602    5    10       0      CG  5:26974854

[606685 rows x 5 columns]
            boxID  6ACCrep3
0      1:10000619  0.000000
1      1:10001679  0.000000
2      1:10002938  0.000000
3       1:1000436  0.000000
4      1:10005021  0.026316
...           ...       ...
57604   5:9995993  0.814815
57605   5:9996636  0.844444
57606   5:9997107  0.804348
57607   5:9998635  0.937500
57608   5:9998667  0.937500

[57609 rows x 2 columns]
         chr  mets  nomets context       boxID


         chr  mets  nomets context       boxID
0          1     0       7      CG      1:1538
1          1     0       5      CG      1:1538
36         1     0      10      CG      1:1538
37         1     0       8      CG      1:1538
71         1     0      10      CG      1:1538
...      ...   ...     ...     ...         ...
4420570    5     7       0      CG  5:26974854
4420571    5     6       0      CG  5:26974854
4420587    5     6       0      CG  5:26974854
4420588    5     4       2      CG  5:26974854
4420602    5     9       1      CG  5:26974854

[537343 rows x 5 columns]
            boxID   6JArep3
0      1:10000619  0.000000
1      1:10001679  0.025641
2      1:10002938  0.000000
3       1:1000436  0.000000
4      1:10005021  0.000000
...           ...       ...
57231   5:9995993  0.896552
57232   5:9996636  0.857143
57233   5:9997107  0.791667
57234   5:9998635  0.735294
57235   5:9998667  0.735294

[57236 rows x 2 columns]
         chr  mets  nomets context       boxID


         chr  mets  nomets context       boxID
0          1     0       8      CG      1:1538
1          1     0       7      CG      1:1538
36         1     0       8      CG      1:1538
37         1     0      13      CG      1:1538
71         1     0      10      CG      1:1538
...      ...   ...     ...     ...         ...
4420571    5     8       3      CG  5:26974854
4420587    5     8       0      CG  5:26974854
4420588    5    10       1      CG  5:26974854
4420601    5     6       2      CG  5:26974854
4420602    5     9       3      CG  5:26974854

[575482 rows x 5 columns]
            boxID  24Mockrep3
0      1:10000619    0.000000
1      1:10001679    0.000000
2      1:10002938    0.000000
3       1:1000436    0.020000
4      1:10005021    0.000000
...           ...         ...
57467   5:9995993    0.880000
57468   5:9996636    0.875000
57469   5:9997107    0.765957
57470   5:9998635    0.800000
57471   5:9998667    0.800000

[57472 rows x 2 columns]
         chr  mets  nom

         chr  mets  nomets context       boxID
0          1     0      14      CG      1:1538
1          1     0      10      CG      1:1538
36         1     0      14      CG      1:1538
37         1     0      11      CG      1:1538
71         1     0      12      CG      1:1538
...      ...   ...     ...     ...         ...
4420568    5     8       0      CG  5:26974854
4420570    5     8       0      CG  5:26974854
4420587    5     5       1      CG  5:26974854
4420601    5     5       0      CG  5:26974854
4420602    5     4       1      CG  5:26974854

[591190 rows x 5 columns]
            boxID  1ACCrep3
0      1:10000619  0.000000
1      1:10001679  0.000000
2      1:10002938  0.000000
3       1:1000436  0.000000
4      1:10005021  0.000000
...           ...       ...
57479   5:9995993  0.952381
57480   5:9996636  0.882353
57481   5:9997107  0.689655
57482   5:9998635  0.882353
57483   5:9998667  0.882353

[57484 rows x 2 columns]
         chr  mets  nomets context       boxID


         chr  mets  nomets context       boxID
0          1     0      10      CG      1:1538
1          1     0       9      CG      1:1538
36         1     0       5      CG      1:1538
37         1     0      10      CG      1:1538
72         1     0       6      CG      1:1538
...      ...   ...     ...     ...         ...
4420571    5     7       0      CG  5:26974854
4420587    5     6       1      CG  5:26974854
4420588    5     5       0      CG  5:26974854
4420601    5     4       2      CG  5:26974854
4420602    5     5       1      CG  5:26974854

[578983 rows x 5 columns]
            boxID   1JArep3
0      1:10000619  0.000000
1      1:10001679  0.000000
2      1:10002938  0.000000
3       1:1000436  0.000000
4      1:10005021  0.000000
...           ...       ...
57501   5:9995993  0.918033
57502   5:9996636  0.898551
57503   5:9997107  0.700000
57504   5:9998635  0.800000
57505   5:9998667  0.800000

[57506 rows x 2 columns]
         chr  mets  nomets context       boxID


         chr  mets  nomets context       boxID
2          1     0       6     CHH      1:1538
3          1     0       5     CHH      1:1538
4          1     0       5     CHH      1:1538
5          1     0       5     CHH      1:1538
6          1     0       5     CHH      1:1538
...      ...   ...     ...     ...         ...
4420615    5     2       7     CHH  5:26974854
4420616    5     1       8     CHH  5:26974854
4420617    5     0      10     CHH  5:26974854
4420618    5     1       9     CHH  5:26974854
4420619    5     0      10     CHH  5:26974854

[3204665 rows x 5 columns]
            boxID  6Mockrep3
0      1:10000619   0.000000
1      1:10001679   0.001669
2      1:10002938   0.003155
3       1:1000436   0.000000
4      1:10005021   0.000000
...           ...        ...
58932   5:9995993   0.338462
58933   5:9996636   0.174699
58934   5:9997107   0.269841
58935   5:9998635   0.146435
58936   5:9998667   0.177419

[58937 rows x 2 columns]
         chr  mets  nomets context

         chr  mets  nomets context       boxID
2          1     0       9     CHH      1:1538
3          1     0      12     CHH      1:1538
4          1     0      12     CHH      1:1538
5          1     0      12     CHH      1:1538
6          1     0      12     CHH      1:1538
...      ...   ...     ...     ...         ...
4420615    5     2       8     CHH  5:26974854
4420616    5     0      10     CHH  5:26974854
4420617    5     1       5     CHH  5:26974854
4420618    5     1       6     CHH  5:26974854
4420619    5     0       7     CHH  5:26974854

[3366238 rows x 5 columns]
            boxID  24ACCrep3
0      1:10000619   0.002049
1      1:10001679   0.003738
2      1:10002938   0.004149
3       1:1000436   0.000000
4      1:10005021   0.005736
...           ...        ...
59024   5:9995993   0.222904
59025   5:9996636   0.133705
59026   5:9997107   0.186501
59027   5:9998635   0.206847
59028   5:9998667   0.229197

[59029 rows x 2 columns]
         chr  mets  nomets context

         chr  mets  nomets context       boxID
11         1     0      11     CHH      1:1538
12         1     0      10     CHH      1:1538
13         1     0      11     CHH      1:1538
17         1     0      11     CHH      1:1538
18         1     0      11     CHH      1:1538
...      ...   ...     ...     ...         ...
4420557    5     1       4     CHH  5:26974854
4420559    5     0       5     CHH  5:26974854
4420560    5     1       4     CHH  5:26974854
4420562    5     0       5     CHH  5:26974854
4420563    5     0       5     CHH  5:26974854

[2943561 rows x 5 columns]
            boxID  24JArep3
0      1:10000619  0.003663
1      1:10001679  0.000000
2      1:10002938  0.009585
3       1:1000436  0.002469
4      1:10005021  0.001852
...           ...       ...
58826   5:9995993  0.210000
58827   5:9996636  0.208092
58828   5:9997107  0.137500
58829   5:9998635  0.202454
58830   5:9998667  0.233333

[58831 rows x 2 columns]
         chr  mets  nomets context       boxID

         chr  mets  nomets context       boxID
2          1     0       6     CHH      1:1538
3          1     0       6     CHH      1:1538
4          1     0       6     CHH      1:1538
5          1     0       6     CHH      1:1538
6          1     0       6     CHH      1:1538
...      ...   ...     ...     ...         ...
4420615    5     1       8     CHH  5:26974854
4420616    5     0       9     CHH  5:26974854
4420617    5     0       9     CHH  5:26974854
4420618    5     0      10     CHH  5:26974854
4420619    5     0       9     CHH  5:26974854

[3278066 rows x 5 columns]
            boxID  1Mockrep3
0      1:10000619   0.000000
1      1:10001679   0.000000
2      1:10002938   0.000000
3       1:1000436   0.000000
4      1:10005021   0.001931
...           ...        ...
59005   5:9995993   0.293269
59006   5:9996636   0.228774
59007   5:9997107   0.362069
59008   5:9998635   0.183932
59009   5:9998667   0.201942

[59010 rows x 2 columns]
         chr  mets  nomets context

In [24]:
replicateDfPercentageOfMetCs.loc[replicateDfPercentageOfMetCs['boxID'] == '1:1538']

,boxID,1JArep2
2677,1:1538,0.0


original

In [4]:
allthedata = []
for experimentCondition in experimentsClasification:
    listOfDfReplicatesPercentageOfMetCs = []
    listOfDfReplicatesCualitative = []
    listOfNamesReplicates = []
    for replicatePath in experimentsClasification[experimentCondition]:

        replicaname =  experimentCondition+replicatePath.split('/')[-4]
        replicateDf = loadFileRemoveMtsAndLowMetReport(replicatePath)

        replicateDfPercentageOfMetCs = calculateValueAsPercentageOfMetCs(replicateDf,replicaname)

#         replicateDfCualitative = calculateValueAsCualitative(replicateDf,replicaname)
#         print(replicateDfCualitative)
        listOfNamesReplicates.append(replicaname)
        listOfDfReplicatesPercentageOfMetCs.append(replicateDfPercentageOfMetCs)
#         listOfDfReplicatesCualitative.append(replicateDfCualitative)
        
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['chr','position', 'strand','context','boxID'],
                                                how='outer'), listOfDfReplicatesPercentageOfMetCs)
    print(df_merged)
    df_merged = df_merged.dropna()
    df_merged[experimentCondition] = df_merged[listOfNamesReplicates].mean(axis=1)
    print(df_merged)
    df_merged = df_merged.drop(columns=listOfNamesReplicates)
    allthedata.append(df_merged)
finaldf = reduce(lambda  left,right: pd.merge(left,right,on=['chr','position', 'strand','context','boxID'],
                                                how='outer'), allthedata)
finaldf = finaldf.dropna()

         chr  position strand context       boxID  6ACCrep3  6ACCrep1  \
0          1      1438      +      CG      1:1538       0.0       0.0   
1          1      1439      -      CG      1:1538       0.0       NaN   
2          1      1444      -     CHH      1:1538       0.0       NaN   
3          1      1449      -     CHH      1:1538       0.0       NaN   
4          1      1450      -     CHH      1:1538       0.0       NaN   
...      ...       ...    ...     ...         ...       ...       ...   
4169007    5  26967861      -      CG  5:26967769       NaN       NaN   
4169008    5  26967865      -     CHG  5:26967769       NaN       NaN   
4169009    5  26967867      -     CHH  5:26967769       NaN       NaN   
4169010    5  26974913      +     CHH  5:26974854       NaN       NaN   
4169011    5  26974918      +     CHH  5:26974854       NaN       NaN   

         6ACCrep2  
0        0.000000  
1        0.000000  
2        0.000000  
3        0.000000  
4        0.000000  
...

         chr  position strand context       boxID  24ACCrep3  24ACCrep1  \
0          1      1438      +      CG      1:1538   0.000000   0.000000   
1          1      1439      -      CG      1:1538   0.000000   0.000000   
2          1      1444      -     CHH      1:1538   0.000000   0.000000   
3          1      1449      -     CHH      1:1538   0.000000   0.000000   
4          1      1450      -     CHH      1:1538   0.000000   0.000000   
...      ...       ...    ...     ...         ...        ...        ...   
3968949    5  26974940      -     CHH  5:26974854   0.200000   0.214286   
3968950    5  26974941      -     CHH  5:26974854   0.000000   0.000000   
3968951    5  26974943      +     CHH  5:26974854   0.166667   0.000000   
3968952    5  26974945      +     CHH  5:26974854   0.142857   0.000000   
3968953    5  26974952      +     CHH  5:26974854   0.000000   0.000000   

         24ACCrep2     24ACC  
0         0.000000  0.000000  
1         0.000000  0.000000  
2     

         chr  position strand context       boxID  1Mockrep3  1Mockrep1  \
0          1      1438      +      CG      1:1538        0.0        0.0   
1          1      1439      -      CG      1:1538        0.0        NaN   
2          1      1444      -     CHH      1:1538        0.0        NaN   
3          1      1449      -     CHH      1:1538        0.0        NaN   
4          1      1450      -     CHH      1:1538        0.0        NaN   
...      ...       ...    ...     ...         ...        ...        ...   
4149736    5  26965670      -      CG  5:26965576        NaN        NaN   
4149737    5  26965671      -     CHG  5:26965576        NaN        NaN   
4149738    5  26965672      -     CHH  5:26965576        NaN        NaN   
4149739    5  26965674      -     CHH  5:26965576        NaN        NaN   
4149740    5  26967680      +     CHH  5:26967769        NaN        NaN   

         1Mockrep2  
0              0.0  
1              0.0  
2              0.0  
3              

In [ ]:
finaldf

In [ ]:
finaldf.to_csv('/home/joaquin/projects/methylation/methylation_jose/met_site/bisulfite.tsv',sep='\t', index=False)

In [6]:
import pandas as pd 
peaksfile = '/home/joaquin/projects/methylation/data/commonData/arabidopsisThaliana/intersect/allThePossiblePeaksnine.bed'

peaks = pd.read_csv(peaksfile, sep='\t', names=['chrm','start','end','boxID'])


df = finaldf[(finaldf['context'] != 'CG')]
metadata = df.iloc[:,4:5]
values = df.iloc[:,5:]

merged = metadata.merge(values, left_index=True, right_index=True, how='inner')
meandf = merged.groupby('boxID', as_index=False).mean()
peaks = peaks.merge(meandf, on='boxID', how='outer')
peaks = peaks[['chrm','start','end','boxID','1Mock','1JA','1ACC','6Mock','6JA','6ACC','24Mock','24JA','24ACC']]



# peaks.to_csv('/home/joaquin/projects/methylation/data/bisulfite_quick_and_dirty_rep1_rep2/CG_BSMet_remove_nan_local_mode.tsv',sep='\t', index=False)

# peaks.to_csv('CHG_BSMet_mean.tsv',sep='\t', index=False)

peaks.to_csv('/home/joaquin/projects/methylation/data/bisulfite_quick_and_dirty_rep1_rep2/non-CG_BSMet_remove_nan_local_mode.tsv.tsv',sep='\t', index=False)
# meandf['chr'], meandf['start'], meandf['end'] =  zip(*meandf['boxID'].map(lambda x: getpositions(x)))

In [ ]:
df

In [ ]:
metadata

In [ ]:
values

In [ ]:
merged

In [ ]:
meandf

In [ ]:
peaks